In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time
import sys
import copy
#Quick fudge to make import from ../Scripts work
import sys
sys.path.append('../Scripts')



# used for weighted percentiles.  
from PostProcessingScripts import * 
import pandas as pd


# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))


%matplotlib inline
import pesummary
from pesummary.io import read
print(pesummary.__version__)
import h5py
# %config InlineBackend.figure_format = 'retina'



../Scripts/PostProcessingScripts.py:25: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  matplotlib.rcParams['text.latex.preamble'] = [r'\boldmath']
../Scripts/PostProcessingScripts.py:44: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
../Scripts/PostProcessingScripts.py:45: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  matplotlib.rcParams['text.latex.preamble'] = [r'\boldmath']


0.13.0


##   code to create csv files with credible intervals possible GWTC-2 BHNS detections
To make this original code work, you will have to download the hdf5 files from the GWTC-2 catalog that belongs to https://arxiv.org/abs/2010.14527
This script just creates csv files that summarize the credible intervals of interest for the possible BHNS binaries from GWTC-2. It is probably the most easiest to directly use those CSV files. 

The GWTC-2 data is available from https://dcc.ligo.org/LIGO-P2000223/public 

In [9]:


allBHNS = ['GW190425', 'GW190814', 'GW190426_152155', 'GW200105_162426', 'GW200115_042309','GW190917_114630' ]

BHNSsGWTC2 = ['GW190425', 'GW190814', 'GW190426_152155'] 
GWTC2b = ['GW200105_162426', 'GW200115_042309']
GWTC2_1 = ['GW190917_114630']


def obtainGWTC1_MassAndWeight(dirGWTC='/Volumes/Andromeda/GWTC-1_LVK_catalog/GWTC-2_sample_release/', dfCSVname='/Users/floorbroekgaarden/Projects/BlackHole-NeutronStar/csvFiles/GWTC_Quantiles/'):
    
    print('calculating credible intervals for possible GWTC-2 BHNS systems')
    
    for ind_BHNS, BHNS_ in enumerate(allBHNS):
    
        if BHNS_ in BHNSsGWTC2:
            print(BHNS_)
            BHNS_file = dirGWTC + BHNS_ +'_comoving.h5'
            BHNS = h5.File(BHNS_file, 'r')

            data = read(BHNS_file)
            samples_dict = data.samples_dict
            posterior_samples = samples_dict['PublicationSamples']
            parameters = sorted(list(posterior_samples.keys()))

            mass_1_source = posterior_samples['mass_1_source']
            mass_2_source = posterior_samples['mass_2_source']
            total_mass_source = posterior_samples['total_mass_source']
            chirp_mass_source= posterior_samples['chirp_mass_source']
            # our mass ratio is defined the other way around
            mass_ratio= 1./posterior_samples['mass_ratio']
            print('mass ratios', BHNS_)
            print(mass_ratio)
            print(mass_1_source/mass_2_source)
            


        elif BHNS_ in GWTC2b:
            GW_ = BHNS_ 
            print(GW_)
            GW_file = '/Volumes/Andromeda/GWTC-1_LVK_catalog/GWTC-2_sample_release/' + GW_ +'_posterior_samples_v2.h5'
            GW = h5.File(GW_file, 'r')
            print(GW['C01:Combined_PHM_high_spin']['posterior_samples']['mass_1_source'][...].squeeze())

            print(GW.keys())
#             data = read(GW_file)
#             samples_dict = data.samples_dict
#             posterior_samples = samples_dict['PublicationSamples']
    #             parameters = sorted(list(posterior_samples.keys()))
            posterior_samples = GW['C01:Combined_PHM_high_spin']['posterior_samples']
            mass_1_source     = posterior_samples['mass_1_source'][...].squeeze()
            mass_2_source     = posterior_samples['mass_2_source'][...].squeeze()
            total_mass_source = posterior_samples['total_mass_source'][...].squeeze()
            chirp_mass_source = posterior_samples['chirp_mass_source'][...].squeeze()

            
            # mass ratio
            mass_ratio = mass_1_source / mass_2_source  # is the same as: posterior_samples['inverted_mass_ratio']
            print('mass ratios', BHNS_)
            print(mass_ratio)
            print(mass_1_source/mass_2_source)
                        
            


        elif BHNS_ in GWTC2_1:
            GW_ = BHNS_
            print(GW_)
            GW_file = '/Volumes/Andromeda/GWTC-1_LVK_catalog/GWTC-2_sample_release/IGWN-GWTC2p1-v1-'+GW_ +'_PEDataRelease.h5'
            GW = h5.File(GW_file, 'r')
#             print(GW[['PrecessingSpinIMRHM']]['posterior_samples']['mass_1_source'][...].squeeze())

            print(GW.keys())
#             data = read(GW_file)
#             samples_dict = data.samples_dict
#             posterior_samples = samples_dict['PublicationSamples']
    #             parameters = sorted(list(posterior_samples.keys()))
            posterior_samples = GW['PrecessingSpinIMRHM']['posterior_samples']
            mass_1_source     = posterior_samples['mass_1_source'][...].squeeze()
            mass_2_source     = posterior_samples['mass_2_source'][...].squeeze()
            total_mass_source = posterior_samples['total_mass_source'][...].squeeze()
            chirp_mass_source = posterior_samples['chirp_mass_source'][...].squeeze()

            
            # mass ratio
            mass_ratio = mass_1_source / mass_2_source  # is the same as: posterior_samples['inverted_mass_ratio']
            print('mass ratios', BHNS_)
            print(mass_ratio)
            print(mass_1_source/mass_2_source)
            
            
            

            
            

        # prepare DataFrame 
        xvarHeaders = ['mass_1_source', 'mass_2_source', \
                        'total_mass_source', 'chirp_mass_source', 'symmetric_mass_ratio']

        xvarUnits = ['Msun', 'Msun', 'Msun', 'Msun', '#']

        # quantiles that I want to know
        y_quantiles  =          [ 0.005, 0.05,   0.5,    0.95,  0.995]
        indexnames   = ['unit', '0.005', '0.05',  '0.5',  '0.95', '0.995']
        # nr of rows and columns that will be used:
        ncol_var = len(xvarHeaders)   
        ncol_Rate_det = 1

        nrows = len(y_quantiles) + 1 # +1 for units (see below)
        # store variables, and Observed and intrinsic rates for all MSSFR variations:
        ncol = ncol_var #* (ncol_MSSFR) # for each MSSFR I want to give quantiles for each xparam 
        df_placeholder = np.zeros((nrows, ncol)) # will be filled in loop: 

        headernames=[]
        units=[]
        for ind_s, ss in enumerate(xvarHeaders):
            sss = ss 
            headernames.append(sss)
            units.append(xvarUnits[ind_s])

        # store dataFrame with zeros that we will fill on the go:
        dfw = pd.DataFrame(data=df_placeholder, columns=headernames, index=indexnames)   
        # add units at first row (index=0)
        dfw.iloc[0]=units        



        # obtain BH and NS masses from combining GW posteriors.
        # by definition LIGO gives Most massive and least massive. 
        xvarlist = [mass_1_source, mass_2_source, total_mass_source, chirp_mass_source, mass_ratio]


        # calculate quantiles with bootstrapping (if Nrepeats>1)
        for ind_xvar, xvar in enumerate(xvarlist):


                # calculate quantiles (credible intervals) from data 
                xvar_quantiles = weighted_quantile(values=xvar, quantiles=y_quantiles, \
                     sample_weight=np.ones_like(xvar))

                dfw_key = xvarHeaders[ind_xvar] 
                dfw[dfw_key][1:] = xvar_quantiles


        dfCSVname_ = dfCSVname + 'CredibleIntervals_' + BHNS_  + '.csv' 
        dfw.to_csv(dfCSVname_) 


        print()
    print('finished')



      
    return

In [10]:
obtainGWTC1_MassAndWeight(dirGWTC='/Volumes/Andromeda/GWTC-1_LVK_catalog/GWTC-2_sample_release/', \
                          dfCSVname='/Users/floorbroekgaarden/Projects/Github/BlackHole-NeutronStar/csvFiles/GWTC_Quantiles/')






calculating credible intervals for possible GWTC-2 BHNS systems
GW190425


2021-08-29  23:04:36 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default
2021-08-29  23:04:36 PESummary WARNING : Could not find delta_f in input file and one was not passed from the command line. Using 0.00390625Hz as default
2021-08-29  23:04:36 PESummary WARNING : Could not find minimum frequency in input file and one was not passed from the command line. Using 20.0Hz as default
2021-08-29  23:04:36 PESummary INFO    : chi_p = 0 for all samples. Treating this as a non-precessing system
2021-08-29  23:04:36 PESummary INFO    : Skipping remnant calculations as tidal deformability parameters found in the posterior table.
2021-08-29  23:04:36 PESummary WARNING : Could not find reference_frequency in input file. Using 20Hz as default
2021-08-29  23:04:41 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default
2021-08-29  23:04:41 PESummary 

mass ratios GW190425
[1.02836824 1.26139008 1.1907685  ... 1.51270026 1.18272038
 1.33072726]
[1.02836824 1.26139008 1.1907685  ... 1.51270026 1.18272038
 1.33072726]

GW190814


2021-08-29  23:05:47 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default
2021-08-29  23:05:47 PESummary WARNING : Could not find delta_f in input file and one was not passed from the command line. Using 0.00390625Hz as default
2021-08-29  23:05:47 PESummary WARNING : Could not find minimum frequency in input file and one was not passed from the command line. Using 20.0Hz as default
2021-08-29  23:06:07 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default
2021-08-29  23:06:07 PESummary WARNING : Could not find delta_f in input file and one was not passed from the command line. Using 0.00390625Hz as default
2021-08-29  23:06:07 PESummary WARNING : Could not find minimum frequency in input file and one was not passed from the command line. Using 20.0Hz as default


mass ratios GW190814
[9.58651545 9.27642139 9.11610399 ... 8.82864351 8.82074926
 8.86890361]
[9.58651545 9.27642139 9.11610399 ... 8.82864351 8.82074926
 8.86890361]

GW190426_152155
mass ratios GW190426_152155
[3.75097054 4.47442311 3.5266919  ... 2.55547708 5.02951608
 4.14227451]
[3.75097054 4.47442311 3.5266919  ... 2.55547708 5.02951608
 4.14227451]

GW200105_162426
[8.37598737 8.73262229 8.41748688 ... 8.98889051 9.25655911
 9.00633161]
<KeysViewHDF5 ['C01:Combined_NSBH_low_spin', 'C01:Combined_PHM_high_spin', 'C01:Combined_PHM_low_spin', 'C01:PhenomNSBH_low_spin', 'C01:PhenomXAS_a1_lessthan_0point5_low_spin', 'C01:PhenomXAS_low_spin', 'C01:PhenomXHM_a1_lessthan_0point5_low_spin', 'C01:PhenomXHM_high_spin', 'C01:PhenomXHM_low_spin', 'C01:PhenomXPHM_high_spin', 'C01:PhenomXPHM_low_spin', 'C01:SEOBNRv4HM_a1_lessthan_0point5_low_spin', 'C01:SEOBNRv4HM_high_spin', 'C01:SEOBNRv4HM_low_spin', 'C01:SEOBNRv4NSBH_a1_lessthan_0point5_low_spin', 'C01:SEOBNRv4NSBH_low_spin', 'C01:SEOBNRv4PH

In [ ]:






BHNSsGWTC2 = ['GW190425', 'GW190814', 'GW190426_152155'] 

def obtainGWTC1_MassAndWeight(dirGWTC='/Volumes/Andromeda/GWTC-1_LVK_catalog/GWTC-2_sample_release/', dfCSVname='/Users/floorbroekgaarden/Projects/BlackHole-NeutronStar/csvFiles/GWTC_Quantiles/'):
    
    print('calculating credible intervals for possible GWTC-2 BHNS systems')
    for ind_BHNS, BHNS_ in enumerate(BHNSsGWTC2):
        print(BHNS_)
        BHNS_file = dirGWTC + BHNS_ +'_comoving.h5'
        BHNS = h5.File(BHNS_file, 'r')

        data = read(BHNS_file)
        samples_dict = data.samples_dict
        posterior_samples = samples_dict['PublicationSamples']
        parameters = sorted(list(posterior_samples.keys()))

        mass_1_source = posterior_samples['mass_1_source']
        mass_2_source = posterior_samples['mass_2_source']
        total_mass_source = posterior_samples['total_mass_source']
        chirp_mass_source= posterior_samples['chirp_mass_source']
        # our mass ratio is defined the other way around
        symmetric_mass_ratio= 1./posterior_samples['mass_ratio']



        # prepare DataFrame 
        xvarHeaders = ['mass_1_source', 'mass_2_source', \
                        'total_mass_source', 'chirp_mass_source', 'symmetric_mass_ratio']

        xvarUnits = ['Msun', 'Msun', 'Msun', 'Msun', '#']

        # quantiles that I want to know
        y_quantiles  =          [ 0.005, 0.05,   0.5,    0.95,  0.995]
        indexnames   = ['unit', '0.005', '0.05',  '0.5',  '0.95', '0.995']
        # nr of rows and columns that will be used:
        ncol_var = len(xvarHeaders)   
        ncol_Rate_det = 1

        nrows = len(y_quantiles) + 1 # +1 for units (see below)
        # store variables, and Observed and intrinsic rates for all MSSFR variations:
        ncol = ncol_var #* (ncol_MSSFR) # for each MSSFR I want to give quantiles for each xparam 
        df_placeholder = np.zeros((nrows, ncol)) # will be filled in loop: 

        headernames=[]
        units=[]
        for ind_s, ss in enumerate(xvarHeaders):
            sss = ss 
            headernames.append(sss)
            units.append(xvarUnits[ind_s])

        # store dataFrame with zeros that we will fill on the go:
        dfw = pd.DataFrame(data=df_placeholder, columns=headernames, index=indexnames)   
        # add units at first row (index=0)
        dfw.iloc[0]=units        



        # obtain BH and NS masses from combining GW posteriors.
        # by definition LIGO gives Most massive and least massive. 
        xvarlist = [mass_1_source, mass_2_source, total_mass_source, chirp_mass_source, symmetric_mass_ratio]


        # calculate quantiles with bootstrapping (if Nrepeats>1)
        for ind_xvar, xvar in enumerate(xvarlist):


                # calculate quantiles (credible intervals) from data 
                xvar_quantiles = weighted_quantile(values=xvar, quantiles=y_quantiles, \
                     sample_weight=np.ones_like(xvar))

                dfw_key = xvarHeaders[ind_xvar] 
                dfw[dfw_key][1:] = xvar_quantiles


        dfCSVname_ = dfCSVname + 'CredibleIntervals_' + BHNS_  + '.csv' 
        dfw.to_csv(dfCSVname_) 


        print()
    print('finished')



      
    return